In [5]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
# 1. Data Preparation and Loading
# create **known** parameters
weight = 0.7
bias = 0.3

# create 
start = 0
end = 1
steps = 0.02
X = torch.arange(start, end, steps).unsqueeze(dim=1) # add 1 dims

y = weight * X + bias
X[:10]
y[:10]

tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])

In [30]:
len(X), len(y)

(50, 50)

In [33]:
## Spliting data into training and test sets    
# traing, validation and test set

# creating training set
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_test)

10